In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/API')
print(os.getcwd())

/content/drive/MyDrive/API


In [3]:
!pip install kaggle

In [4]:
import json
with open('kaggle.json', 'r') as file:
  data = json.load(file)

# print(data)

In [5]:
os.environ['KAGGLE_USERNAME'] = data['username']
os.environ['KAGGLE_KEY'] = data['key']

In [6]:
import shutil
import regex as re
import kaggle
import kagglehub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
num_cores = os.cpu_count()
os.environ['TF_INTRA_OP_PARALLELISM'] = str(num_cores)
tf.config.optimizer.set_jit(True)

In [ ]:
# # os.mkdir("kaggle")
# # shutil.copy2("/content/drive/MyDrive/API/kaggle.json", "/content/drive/MyDrive/API/kaggle/kaggle.json")
# os.chdir('/content/drive/MyDrive/API/kaggle')
# !chmod 600 /root/.kaggle/kaggle.json
# print(os.getcwd())

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
/content/drive/MyDrive/API/kaggle


In [ ]:
shutil.copy2("/content/drive/MyDrive/API/kaggle.json", "/content/drive/MyDrive/API/kaggle/kaggle.json")

'/content/drive/MyDrive/API/kaggle/kaggle.json'

In [ ]:
os.getcwd()

'/content/drive/MyDrive/API'

In [ ]:
os.chdir('/content/drive/MyDrive/Dataset/Movies')
print(os.getcwd())

/content/drive/MyDrive/Dataset/Movies


In [ ]:
# Download latest version
path = kagglehub.dataset_download("rounakbanik/the-movies-dataset")

print("Path to dataset files:", path)

100%|██████████| 228M/228M [00:02<00:00, 111MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rounakbanik/the-movies-dataset/versions/7


In [ ]:
source_dir = "/root/.cache/kagglehub/datasets/rounakbanik/the-movies-dataset/versions/7"
destination_dir = "/content/drive/MyDrive/Dataset/Movies"

In [ ]:
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)
print(f"Konten dari {source_dir} berhasil disalin ke {destination_dir}")

Konten dari /root/.cache/kagglehub/datasets/rounakbanik/the-movies-dataset/versions/7 berhasil disalin ke /content/drive/MyDrive/Dataset/Movies


# Load Dataset & EDA

### credits dataset

In [7]:
credits = pd.read_csv('/content/drive/MyDrive/Dataset/Movies/credits.csv')
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [ ]:
print(credits.isna().sum())
credits.info()

cast    0
crew    0
id      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


### keyword dataset

In [ ]:
keyword = pd.read_csv('/content/drive/MyDrive/Dataset/Movies/keywords.csv')
keyword.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
print(keyword.isna().sum())
keyword.info()

id          0
keywords    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


### movies metadata

In [8]:
movies_metadata = pd.read_csv('/content/drive/MyDrive/Dataset/Movies/movies_metadata.csv')
# movies_metadata["id"].astype(int)
movies_metadata.head()

<ipython-input-8-e99e9f5f681f>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('/content/drive/MyDrive/Dataset/Movies/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [9]:
print(movies_metadata.isna().sum())
movies_metadata.info()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45

In [10]:
id_rgx = r'^\d{4}-\d{2}-\d{2}$'
movies_metadata[movies_metadata['id'].str.match(id_rgx)]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
originial_title_rgx = r'^\[{'
movies_metadata[movies_metadata['original_title'].str.match(originial_title_rgx)]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
movies_metadata = movies_metadata[movies_metadata['id'].str.match(id_rgx) == False]
movies_metadata = movies_metadata[movies_metadata['original_title'].str.match(originial_title_rgx) == False]

In [13]:
movies_metadata['id'] = movies_metadata['id'].astype('int')

In [14]:
movies_metadata[movies_metadata['id'] == 862]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [15]:
movies_metadata[movies_metadata['title'].isna()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19729,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,82663,tt0113002,en,Midnight Man,British soldiers force a recently captured IRA...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29502,False,"{'id': 122661, 'name': 'Mardock Scramble Colle...",0,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...",http://m-scramble.jp/exhaust/,122662,tt2423504,ja,マルドゥック・スクランブル 排気,Third film of the Mardock Scramble series.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35586,False,NaN,0,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 28,...",NaN,249260,tt2622826,en,Avalanche Sharks,A group of skiers are terrorized during spring...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# hsgdjhsa = movies_metadata['genres']

def parse_genres(json_str):
  try:
    genres = json.loads(json_str.replace("'", '"'))
    return [genre['name'] for genre in genres]
  except (TypeError, json.JSONDecodeError):
        return []

movies_metadata['genres'] = movies_metadata['genres'].apply(parse_genres)
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
all_genres = [genre for genre_lists in movies_metadata['genres'] for genre in genre_lists]
count_genre = pd.Series(all_genres).value_counts()
count_genre.head()

,count
Drama,20265
Comedy,13182
Thriller,7624
Romance,6735
Action,6596


### ratings dataset

In [17]:
ratings = pd.read_csv('/content/drive/MyDrive/Dataset/Movies/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


In [ ]:
ratings_small = pd.read_csv('/content/drive/MyDrive/Dataset/Movies/ratings_small.csv')
ratings_small

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [ ]:
ratings_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


### links dataset

In [18]:
links = pd.read_csv('/content/drive/MyDrive/Dataset/Movies/links.csv')
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
45838,176269,6209470,439050.0
45839,176271,2028550,111109.0
45840,176273,303758,67758.0
45841,176275,8536,227506.0


In [ ]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  45843 non-null  int64  
 1   imdbId   45843 non-null  int64  
 2   tmdbId   45624 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.0 MB


In [19]:
links  = pd.merge(links, movies_metadata[['id', 'title']], left_on='tmdbId', right_on='id')
links = links[['movieId', 'imdbId', 'tmdbId', 'title']]
links['tmdbId'] = links['tmdbId'].astype('int')
links

,movieId,imdbId,tmdbId,title
0,1,114709,862,Toy Story
1,2,113497,8844,Jumanji
2,3,113228,15602,Grumpier Old Men
3,4,114885,31357,Waiting to Exhale
4,5,113041,11862,Father of the Bride Part II
...,...,...,...,...
45520,176269,6209470,439050,Subdue
45521,176271,2028550,111109,Century of Birthing
45522,176273,303758,67758,Betrayal
45523,176275,8536,227506,Satan Triumphant


In [ ]:
links_small = pd.read_csv('/content/drive/MyDrive/Dataset/Movies/links_small.csv')
links_small

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


In [ ]:
links_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [ ]:
links_small.join(movies_metadata.set_index('id'), on='movieId')

,movieId,imdbId,tmdbId,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,1,114709,862.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,113497,8844.0,False,NaN,0,"[Drama, Crime]",NaN,tt0094675,fi,...,1988-10-21,0.0,69.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}, {'iso_6...",Released,NaN,Ariel,False,7.1,44.0
2,3,113228,15602.0,False,NaN,0,"[Drama, Comedy]",NaN,tt0092149,fi,...,1986-10-16,0.0,76.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Shadows in Paradise,False,7.1,35.0
3,4,114885,31357.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,113041,11862.0,False,NaN,4000000,"[Crime, Comedy]",NaN,tt0113101,en,...,1995-12-09,4300000.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6.5,539.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,162672,3859980,402672.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9121,163056,4262980,315011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9122,163949,2531318,391698.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9123,164977,27660,137608.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


## Preprocessing

In [20]:
df_merge = pd.merge(links, ratings, left_on='tmdbId', right_on='movieId')
df_merge

,movieId_x,imdbId,tmdbId,title,userId,movieId_y,rating,timestamp
0,1,114709,862,Toy Story,1923,862,3.0,858335006
1,1,114709,862,Toy Story,2103,862,5.0,946044912
2,1,114709,862,Toy Story,5380,862,1.0,878941641
3,1,114709,862,Toy Story,6177,862,4.0,859415226
4,1,114709,862,Toy Story,6525,862,4.0,857388995
...,...,...,...,...,...,...,...,...
11439770,176271,2028550,111109,Century of Birthing,33940,111109,2.5,1405878785
11439771,176271,2028550,111109,Century of Birthing,172224,111109,3.0,1399502972
11439772,176271,2028550,111109,Century of Birthing,210792,111109,3.0,1467090449
11439773,176271,2028550,111109,Century of Birthing,225396,111109,3.5,1399302912


In [21]:
df = df_merge.drop(['movieId_y'], axis=1)
df.head()

,movieId_x,imdbId,tmdbId,title,userId,rating,timestamp
0,1,114709,862,Toy Story,1923,3.0,858335006
1,1,114709,862,Toy Story,2103,5.0,946044912
2,1,114709,862,Toy Story,5380,1.0,878941641
3,1,114709,862,Toy Story,6177,4.0,859415226
4,1,114709,862,Toy Story,6525,4.0,857388995


In [ ]:
df[df['userId'] == np.min(df['userId'])]

,movieId_x,imdbId,tmdbId,title,userId,rating,timestamp
256966,306,111495,110,Three Colors: Red,1,1.0,1425941529
539541,539,108160,858,Sleepless in Seattle,1,5.0,1425941523
1690638,1457,119141,1968,Fools Rush In,1,4.0,1425942148
2343877,2209,29811,2762,Young and Innocent,1,4.5,1425941300
2943547,2731,53198,147,The 400 Blows,1,4.5,1425942435
8429005,49651,479143,1246,Rocky Balboa,1,5.0,1425941556
8824726,53974,762114,2959,License to Wed,1,4.0,1425941601
10114798,98587,2177511,96821,Caesar Must Die,1,5.0,1425941382
10455031,121113,212235,4226,Shriek If You Know What I Did Last Friday the ...,1,4.0,1425942228
10739651,135827,199813,54503,The Mystery of Chess Boxing,1,3.5,1425941313


In [ ]:
df[df['userId'] == np.max(df['userId'])]

,movieId_x,imdbId,tmdbId,title,userId,rating,timestamp
79009,62,113862,2054,Mr. Holland's Opus,270896,3.0,1257034154
173423,260,76759,11,Star Wars,270896,3.5,1257034215
323477,306,111495,110,Three Colors: Red,270896,5.0,1257031440
524852,500,107614,788,Mrs. Doubtfire,270896,2.5,1257034307
596610,539,108160,858,Sleepless in Seattle,270896,5.0,1257031915
...,...,...,...,...,...,...,...
11016503,144828,24703,3578,The Tunnel,270896,4.5,1257031580
11083744,153290,477038,4979,Windows on Monday,270896,4.5,1257033785
11176203,155796,1865335,58559,Confession of a Child of the Century,270896,5.0,1257031564
11250462,162826,472126,2324,Local Color,270896,4.0,1257034135


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11439775 entries, 0 to 11439774
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId_x  int64  
 1   imdbId     int64  
 2   tmdbId     int64  
 3   title      object 
 4   userId     int64  
 5   rating     float64
 6   timestamp  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 611.0+ MB


In [23]:
os.chdir('/content/drive/MyDrive/Dataset/Movies')
print(os.getcwd())

/content/drive/MyDrive/Dataset/Movies


In [24]:
df.to_csv('df.csv', index=False)

In [ ]:
user_id = df['userId'].values
movie_id = df['movieId_x'].values
rating = df['rating'].values

In [ ]:
X_train_user, X_test_user, X_train_item, X_test_item, y_train, y_test = train_test_split(
    user_id, movie_id, rating,
    test_size=0.2, random_state=42)

In [ ]:
X_train_user = np.array(X_train_user).reshape(-1, 1)
X_train_item = np.array(X_train_item).reshape(-1, 1)
y_train = np.array(y_train).reshape(-1, 1)

In [ ]:
X_test_user = np.array(X_test_user).reshape(-1, 1)
X_test_item = np.array(X_test_item).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

# Build Model NCF & Training Model

In [ ]:
embedding_dim = 16
num_users = np.max(user_id) + 1
num_items = np.max(movie_id) + 1

user_input = Input(shape=(1,), name="user_input")
item_input = Input(shape=(1,), name="item_input")

user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name="user_embedding")(user_input)
item_embedding = Embedding(input_dim=num_items, output_dim=embedding_dim, name="item_embedding")(item_input)

user_vector = Flatten(name="user_vector")(user_embedding)
item_vector = Flatten(name="item_vector")(item_embedding)

# layer gmf
gmf_vector = tf.keras.layers.multiply([user_vector, item_vector])

# layer mlp
mlp_vector = Concatenate()([user_vector, item_vector])
mlp_vector = Dense(units=64, activation='relu')(mlp_vector)
mlp_vector = Dense(units=64, activation='relu')(mlp_vector)
mlp_vector = Dense(units=32, activation='relu')(mlp_vector)
mlp_vector = Dense(units=16, activation='relu')(mlp_vector)
mlp_vector = Dropout(rate=0.2)(mlp_vector)

ncf_layer = Concatenate()([gmf_vector, mlp_vector])
output = Dense(units=1, activation='linear')(ncf_layer)

model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ item_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_embedding            │ (None, 1, 16)          │      4,334,352 │ user_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ item_embedding            │ (None, 1, 16)          │      2,820,352 │ item_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_vector (Flatten)     │ (None, 16)             │              0 │ user_embedding[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ item_vector (Flatten)     │ (None, 16)             │              0 │ item_embedding[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 32)             │              0 │ user_vector[0][0],     │
│ (Concatenate)             │                        │                │ item_vector[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 64)             │          2,112 │ concatenate_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 64)             │          4,160 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 32)             │          2,080 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 16)             │            528 │ dense_11[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply_2 (Multiply)     │ (None, 16)             │              0 │ user_vector[0][0],     │
│                           │                        │                │ item_vector[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 16)             │              0 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_5             │ (None, 32)             │              0 │ multiply_2[0][0],      │
│ (Concatenate)             │                        │                │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 1)              │             33 │ concatenate_5[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 7,163,617 (27.33 MB)

 Trainable params: 7,163,617 (27.33 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print(X_test_user.shape)  # Harus (jumlah_sampel, user_embedding_dim)
print(X_test_item.shape)  # Harus (jumlah_sampel, item_embedding_dim)
print(y_test.shape)       # Harus (jumlah_sampel, 1)


(2287955, 1)
(2287955, 1)
(2287955, 1)


In [ ]:
class callbacks(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('loss') < 1.0 and logs.get('val_loss') < 1.0:
      print("Stop Training Model")
      self.model.stop_training = True
      restore_best_weights = True

callbacks = callbacks()

In [ ]:
model.fit(
    x=[X_train_user, X_train_item],
    y=y_train,
    batch_size=128,
    epochs=10,
    validation_data=([X_test_user, X_test_item], y_test),
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/10
71499/71499 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.9389 - mae: 0.7483Stop Training Model
71499/71499 ━━━━━━━━━━━━━━━━━━━━ 4965s 69ms/step - loss: 0.9389 - mae: 0.7483 - val_loss: 0.7278 - val_mae: 0.6524


In [ ]:
df[df['userId'] == 1]

,movieId_x,imdbId,tmdbId,title,userId,rating,timestamp
256966,306,111495,110,Three Colors: Red,1,1.0,1425941529
539541,539,108160,858,Sleepless in Seattle,1,5.0,1425941523
1690638,1457,119141,1968,Fools Rush In,1,4.0,1425942148
2343877,2209,29811,2762,Young and Innocent,1,4.5,1425941300
2943547,2731,53198,147,The 400 Blows,1,4.5,1425942435
8429005,49651,479143,1246,Rocky Balboa,1,5.0,1425941556
8824726,53974,762114,2959,License to Wed,1,4.0,1425941601
10114798,98587,2177511,96821,Caesar Must Die,1,5.0,1425941382
10455031,121113,212235,4226,Shriek If You Know What I Did Last Friday the ...,1,4.0,1425942228
10739651,135827,199813,54503,The Mystery of Chess Boxing,1,3.5,1425941313


## Save Model

In [ ]:
os.getcwd()

'/content/drive/MyDrive/API'

In [ ]:
os.chdir('/content/drive/MyDrive')
print(os.getcwd())

/content/drive/MyDrive


In [ ]:
# os.mkdir('Model')
# os.chdir('/content/drive/MyDrive/Model')
# os.mkdir('Recommendation')
os.chdir('/content/drive/MyDrive/Model/Recommendation')
print(os.getcwd())

/content/drive/MyDrive/Model/Recommendation


In [ ]:
model.save('recommendation_movie_model_ncf.h5')

# Prediction Model

In [ ]:
user = np.array([1]).reshape(-1, 1)
movie = np.array([1]).reshape(-1, 1)

In [ ]:
model.predict([user, movie])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step


array([[3.6523502]], dtype=float32)

In [ ]:
model = tf.keras.models.load_model('recommendation_movie_model_ncf.h5')

In [ ]:
def prediction(user_id, top_n=5):
  all_ids_movie = df['movieId_x'].unique()
  user_id = np.full(len(all_ids_movie), user_id).reshape(-1, 1)
  movie_id = all_ids_movie.reshape(-1, 1)

  prediction_ratings = model.predict([user_id, movie_id])

  recommendation_with_ratings = pd.DataFrame({
      "movieId_x" : all_ids_movie,
      "rating_pred": prediction_ratings.flatten()
  })

  recommendation_with_ratings = recommendation_with_ratings.sort_values(by= "rating_pred", ascending=False)
  recommendation_with_ratings = recommendation_with_ratings.merge(df[["movieId_x", "title"]].drop_duplicates(), on="movieId_x", how="left")
  recommendation = recommendation_with_ratings.drop(["rating_pred"], axis=1)

  return recommendation_with_ratings.head(top_n), recommendation.head(top_n)


In [ ]:
rating_prediction, recommendation = prediction(1, 5)

237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
rating_prediction

,movieId_x,rating_pred,title
0,539,4.243193,Sleepless in Seattle
1,4146,4.203622,The Million Dollar Hotel
2,464,4.194700,Hard Target
3,26338,4.188751,"Cousin, Cousine"
4,6461,4.184836,The Unforgiven


In [ ]:
recommendation

,movieId_x,title
0,539,Sleepless in Seattle
1,4146,The Million Dollar Hotel
2,464,Hard Target
3,26338,"Cousin, Cousine"
4,6461,The Unforgiven
